<a href="https://colab.research.google.com/github/shota74/-/blob/master/CV%E7%89%B9%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet_pytorch
# モジュールのインポート
from time import time
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from efficientnet_pytorch import EfficientNet
from google.colab import drive #インポート

drive.mount('/content/gdrive') #GoogleDriveのマウント
# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

model = EfficientNet.from_pretrained('efficientnet-b4')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 100)
#print(model)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Use CUDA: True
Loaded pretrained weights for efficientnet-b4


In [ ]:
!nvidia-smi

Mon Nov 23 11:11:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    67W / 149W |     11MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
tfms = transforms.Compose([transforms.Resize(224),
                           transforms.RandomHorizontalFlip(p = 0.5),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

tfms_test = transforms.Compose([transforms.Resize(224),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
train_data = torchvision.datasets.CIFAR100(root="./", train=True, transform=tfms, download=True)
test_data = torchvision.datasets.CIFAR100(root="./", train=False, transform=tfms_test, download=True)

print(train_data)
print(test_data)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [ ]:
if use_cuda:
    model.cuda()

#print(model)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [40,70], gamma=0.1)

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 80
epoch_num = 100
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

best_pre = 0
# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()


load_path =  "/content/gdrive/My Drive/機械学習/filename.pth"

model.load_state_dict(torch.load(load_path))
# ネットワークを学習モードへ変更
model.train()

scaler = torch.cuda.amp.GradScaler()
start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        scheduler.step()
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
        
        with torch.cuda.amp.autocast():
            y = model(image)
            loss = criterion(y, label)
        
        #model.zero_grad()
        optimizer.zero_grad()
        #loss.backward()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        #print(loss.item())
        #optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))
    
    model_save_name = 'filename.pth' # 保存ファイル名
    path = "/content/gdrive/My Drive/機械学習/filename.pth" # 保存先pathの設定
    torch.save(model.state_dict(), path) # 保存

    if epoch%10 == 0:
        # ネットワークを評価モードへ変更
        model.eval()

        # 評価の実行
        count = 0
        with torch.no_grad():
            for image, label in test_loader:
                if use_cuda:
                    image = image.cuda()
                    label = label.cuda()
                with torch.cuda.amp.autocast():
                    y = model(image)
                pred = torch.argmax(y, dim=1)
                count += torch.sum(pred == label)
        print("test accuracy: {}".format(count.item() / 10000.))
        if best_pre < (count.item() / 10000):
            best_pre = count.item() / 10000
            model_save = 'filename_best.pth' # 保存ファイル名
            path = "/content/gdrive/My Drive/機械学習/filename_best.pth" # 保存先pathの設定
            torch.save(model.state_dict(), path) # 保存



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch: 1, mean loss: 0.868503481733799, mean accuracy: 0.74628, elapsed_time :1853.481653213501
epoch: 2, mean loss: 0.5952524259090424, mean accuracy: 0.8203, elapsed_time :3705.2299377918243
epoch: 3, mean loss: 0.49164786179065706, mean accuracy: 0.8469, elapsed_time :5557.141652584076
epoch: 4, mean loss: 0.4271341685771942, mean accuracy: 0.8677, elapsed_time :7419.189713001251
epoch: 5, mean loss: 0.3790869579076767, mean accuracy: 0.88044, elapsed_time :9298.979536294937
epoch: 6, mean loss: 0.33827255187034605, mean accuracy: 0.8934, elapsed_time :11224.454279184341
epoch: 7, mean loss: 0.308205536365509, mean accuracy: 0.90382, elapsed_time :13111.497868537903


In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        with torch.cuda.amp.autocast():
            y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))